In [7]:
import json
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json.uie') as frobj:
    data_list = []
    for line in frobj:
        content = json.loads(line.strip())
        data_list.append(content)

In [4]:
data_dict = {}
import json
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json.risk_topic') as frobj:
    for line in frobj:
        content = json.loads(line.strip())
        if content['text'] not in data_dict:
            data_dict[content['text']] = content

In [236]:
data_dict['大妈广场舞乱相到底反映出中国社会怎样的深层次矛盾？']

{'text': '大妈广场舞乱相到底反映出中国社会怎样的深层次矛盾？',
 'topic': [['城市', 0.4229627847671509],
  ['教育/科学', 0.2866935729980469],
  ['社会', 0.10912452638149261],
  ['经济', 0.036604635417461395],
  ['时事政治', 0.02015676721930504]],
 'score_list': {'senti': [['负向', 0.9996033310890198],
   ['正向', 0.00039672438288107514]],
  'bias': [['偏见', 0.03662021830677986], ['正常', 0.9633797407150269]],
  'ciron': [['讽刺', 0.1289997696876526], ['正常', 0.8710001707077026]],
  'intent': [['主观评价/比较/判断', 0.0014891846803948283],
   ['寻求建议/帮助', 0.0010056071914732456],
   ['其它', 0.997505247592926]],
  'offensive': [['冒犯', 0.2526616156101227], ['正常', 0.7473383545875549]],
  'query_risk': [['风险', 0.9999690055847168],
   ['个人信息', 1.959269866347313e-05],
   ['正常', 1.1403845746826846e-05]],
  'teenager': [['不良', 0.7801902294158936], ['正常', 0.21980974078178406]]}}

In [206]:
positive = []
post_key = {}
for d in data_list:
    d_content = data_dict[d['text']]
    if not d['score_list']:
        topic_pred = set([d_content['topic'][0][0], d_content['topic'][1][0]])
        topic = set(['学习', '游戏', '健康', '娱乐', '明星', '体育/运动', '星座', '婚姻', '女性', '职场职业', '情感', '两性', '性生活', '动植物', '小说', '校园生活', '汽车', '影视'])
        if len(topic_pred&topic) > 0:
            post_key[d['text']] = 0
            positive.append(d)

    elif d["score_list"]["情感分类[正向,负向]"][0]['text'] in ['正向'] and d["score_list"]["情感分类[正向,负向]"][0]['probability'] >= 0.8:
        topic_pred = set([d_content['topic'][0][0], d_content['topic'][1][0]])
        topic = set(['心理健康', '国家', '时事政治', '历史', '毒品', '法律', '女权'])
        if len(topic_pred&topic) <= 0:
            positive.append(d)
            post_key[d['text']] = 0
    elif d["score_list"]["情感分类[正向,负向]"][0]['text'] in ['正向'] and d["score_list"]["情感分类[正向,负向]"][0]['probability'] >= 0.2:
        topic_pred = set([d_content['topic'][0][0], d_content['topic'][1][0]])
        topic = set((['学习', '游戏', '健康', '娱乐', '明星', '体育/运动', '星座', '婚姻', '女性', '职场职业', '情感', '两性', '性生活', '动植物', '小说', '校园生活', '汽车', '影视']))
        if len(topic_pred&topic) >= 1:
            positive.append(d)
            post_key[d['text']] = 0
    elif d["score_list"]["情感分类[正向,负向]"][0]['text'] in ['负向']:
        
        topic_pred = set([d_content['topic'][0][0], d_content['topic'][1][0]])
        topic_prob = max([d_content['topic'][0][1], d_content['topic'][1][1]])
        topic = set(['心理健康', '国家', '时事政治', '历史', '毒品', '法律', '女权'])
        if len(topic_pred&topic) >= 1:
            continue
        
        topic_pred = set([d_content['topic'][0][0], d_content['topic'][1][0]])
        topic = set(['学习', '游戏', '健康', '娱乐', '明星', '婚姻',  '体育/运动', '星座', '职场职业', '女性', '情感', '两性', '性生活', '动植物', '小说', '校园生活', '汽车', '影视'])
        
        if len(topic_pred&topic) > 0:
            # if d_content['score_list']['bias'][0][1] >= 0.5 or d_content['score_list']['ciron'][0][1] >= 0.4:
            #     continue
            positive.append(d)
            post_key[d['text']] = 0

In [182]:
d_content['score_list']

{'senti': [['负向', 0.8095175623893738], ['正向', 0.19048240780830383]],
 'bias': [['偏见', 0.02734767086803913], ['正常', 0.9726523160934448]],
 'ciron': [['讽刺', 0.044293444603681564], ['正常', 0.9557065367698669]],
 'intent': [['主观评价/比较/判断', 0.0004939892096444964],
  ['寻求建议/帮助', 0.0010639607207849622],
  ['其它', 0.9984421133995056]],
 'offensive': [['冒犯', 0.4047074019908905], ['正常', 0.5952925682067871]],
 'query_risk': [['风险', 0.9999769926071167],
  ['个人信息', 6.688293979095761e-06],
  ['正常', 1.6362109818146564e-05]],
 'teenager': [['不良', 0.5203346014022827], ['正常', 0.4796653389930725]]}

In [24]:
topic_pred, topic

({'人际交往', '心理健康'}, {'健康'})

In [209]:
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json.positive', 'w') as fwobj:
    for d in positive:
        fwobj.write(json.dumps(d, ensure_ascii=False)+'\n')

In [226]:
'sdfdasjkgf'.split('\t')

['sdfdasjkgf']

In [229]:


left = []
for d in positive:
    text = d['text']
    if len(text.split('\t')) == 1:
        continue
    text, label, score_list, score = text.split('\t')
    d['text'] = text
    left.append(d)

60

In [208]:
with open('/data/albert.xht/raw_chat_corpus/model_risk_xiaoda/query_risk_corpus.json.filter', 'w') as fwobj:
    for d in data_list:
        if d['text'] in post_key:
            continue
        fwobj.write(json.dumps(d, ensure_ascii=False)+'\n')

In [235]:
data_dict['这样的我算是坏女孩吗？']

{'text': '这样的我算是坏女孩吗？',
 'topic': [['情感', 0.21970127522945404],
  ['心理健康', 0.11050224304199219],
  ['校园生活', 0.10356755554676056],
  ['恋爱', 0.08195813745260239],
  ['价值观', 0.07936355471611023]],
 'score_list': {'senti': [['负向', 0.9998934268951416],
   ['正向', 0.00010655508958734572]],
  'bias': [['偏见', 0.09576970338821411], ['正常', 0.9042302966117859]],
  'ciron': [['讽刺', 0.008845423348248005], ['正常', 0.9911545515060425]],
  'intent': [['主观评价/比较/判断', 0.9991254210472107],
   ['寻求建议/帮助', 0.0006391228525899351],
   ['其它', 0.00023552612401545048]],
  'offensive': [['冒犯', 0.15577161312103271], ['正常', 0.8442283868789673]],
  'query_risk': [['风险', 0.9998146891593933],
   ['个人信息', 8.702470950083807e-06],
   ['正常', 0.00017664469487499446]],
  'teenager': [['不良', 0.2951676845550537], ['正常', 0.7048323154449463]]}}

In [225]:
data_dict['运动大量出汗会不会导致痛风？']

{'text': '运动大量出汗会不会导致痛风？',
 'topic': [['体育/运动', 0.7894255518913269],
  ['健康', 0.20242246985435486],
  ['美容/塑身', 0.001110862591303885],
  ['美食/烹饪', 0.0009233857854269445],
  ['女性', 0.0007468782132491469]],
 'score_list': {'senti': [['负向', 0.9998144507408142],
   ['正向', 0.0001855577138485387]],
  'bias': [['偏见', 0.09793330729007721], ['正常', 0.902066707611084]],
  'ciron': [['讽刺', 0.0027007663156837225], ['正常', 0.9972992539405823]],
  'intent': [['主观评价/比较/判断', 1.4931842997611966e-05],
   ['寻求建议/帮助', 0.999981164932251],
   ['其它', 3.908808594133006e-06]],
  'offensive': [['冒犯', 0.36420899629592896], ['正常', 0.635791003704071]],
  'query_risk': [['风险', 0.9985072016716003],
   ['个人信息', 1.4761052625544835e-05],
   ['正常', 0.0014780479250475764]],
  'teenager': [['不良', 0.007095863111317158], ['正常', 0.9929041266441345]]}}